In [75]:
import os, sys, csv, json, re
import numpy as np
import pandas as pd

In [178]:
station_name = 'KLAX'
airport_station_name = 'KLAX'

raw_folder = 'station-raw'
process_folder = 'station-process'
result_folder = 'station-result'

In [179]:
read_file = True
if read_file:
    file_names = os.listdir(raw_folder)
    file_names = ['KLAX.2022-01-01.csv']
    for file_name in file_names:
        if file_name.endswith('csv'):
            station_name = file_name.split('.')[0]
            out_1 = open(os.path.join(process_folder, station_name + '-meta.csv'), 'w')
            out_2 = open(os.path.join(process_folder, station_name + '-raw.csv'), 'w')
            with open(os.path.join(raw_folder, file_name), 'r') as f:
                for line in f:
                    if line.startswith('#') or line.startswith('Station') or line.startswith(','):
                        out_1.write(line)
                    if not line.startswith('#') and not line.startswith(','):
                        out_2.write(line)
            out_1.close()
            out_2.close()

In [180]:
file_name = os.path.join(process_folder, station_name + '-raw.csv')
df_raw = pd.read_csv(file_name, sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
df_raw.columns = map(str.lower,df_raw.columns)
df_raw.columns

C:\Users\trent\AppData\Local\Temp\ipykernel_45436\4179227207.py:2: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(file_name, sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')


Index(['station_id', 'date_time', 'altimeter_set_1', 'air_temp_set_1',
       'dew_point_temperature_set_1', 'relative_humidity_set_1',
       'wind_speed_set_1', 'wind_direction_set_1', 'wind_gust_set_1',
       'sea_level_pressure_set_1', 'weather_cond_code_set_1',
       'cloud_layer_3_code_set_1', 'pressure_tendency_set_1',
       'precip_accum_one_hour_set_1', 'precip_accum_three_hour_set_1',
       'cloud_layer_1_code_set_1', 'cloud_layer_2_code_set_1',
       'precip_accum_six_hour_set_1', 'precip_accum_24_hour_set_1',
       'visibility_set_1', 'metar_set_1', 'air_temp_high_6_hour_set_1',
       'air_temp_low_6_hour_set_1', 'peak_wind_speed_set_1', 'ceiling_set_1',
       'pressure_change_code_set_1', 'air_temp_high_24_hour_set_1',
       'air_temp_low_24_hour_set_1', 'peak_wind_direction_set_1',
       'dew_point_temperature_set_1d', 'cloud_layer_1_set_1d',
       'cloud_layer_3_set_1d', 'cloud_layer_2_set_1d', 'wind_chill_set_1d',
       'weather_summary_set_1d', 'wind_cardin

In [181]:
# processed CSV header to raw CSV header
fields = {'dry_bulb':'air_temp_set_1', 
          'rel_humi':'relative_humidity_set_1', 
          'dew_point':'dew_point_temperature_set_1d', 
          'pressure':'pressure_set_1d',
          'wind_speed':'wind_speed_set_1', 
          'wind_dir':'wind_direction_set_1',
          'wind_chill': 'wind_chill_set_1d',
          'heat_index': 'heat_index_set_1d',
          'cloud_layer_1': 'cloud_layer_1_code_set_1',
#           'precip_accum_24_hour':'precip_accum_24_hour_set_1',
#           'solar_radiation':'solar_radiation_set_1',
#           'cloud_layer_1': 'cloud_layer_1_code_set_1',
#           'weather_condition': 'weather_condition_set_1d',
#           'snow_depth': 'snow_depth_set_1',
         }
fields_to_interp = ['dry_bulb', 'rel_humi', 'dew_point', 'pressure', 'wind_speed']

In [182]:
df_raw.head()

,station_id,date_time,altimeter_set_1,air_temp_set_1,dew_point_temperature_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,sea_level_pressure_set_1,...,cloud_layer_1_set_1d,cloud_layer_3_set_1d,cloud_layer_2_set_1d,wind_chill_set_1d,weather_summary_set_1d,wind_cardinal_direction_set_1d,pressure_set_1d,sea_level_pressure_set_1d,heat_index_set_1d,weather_condition_set_1d
0,KLAX,2018-12-31T12:00:00Z,101252.97,9.0,NaN,81.48,1.54,70.0,NaN,NaN,...,NaN,NaN,NaN,NaN,thin scattered,ENE,100796.17,101262.01,NaN,NaN
1,KLAX,2018-12-31T12:05:00Z,101252.97,9.0,NaN,81.48,2.06,60.0,NaN,NaN,...,NaN,NaN,NaN,NaN,scattered,ENE,100796.17,101262.01,NaN,NaN
2,KLAX,2018-12-31T12:10:00Z,101252.97,9.0,NaN,81.48,2.06,60.0,NaN,NaN,...,NaN,NaN,NaN,NaN,scattered,ENE,100796.17,101262.01,NaN,NaN
3,KLAX,2018-12-31T12:15:00Z,101252.97,9.0,NaN,81.48,2.57,60.0,NaN,NaN,...,NaN,NaN,NaN,NaN,scattered,ENE,100796.17,101262.01,NaN,NaN
4,KLAX,2018-12-31T12:20:00Z,101219.11,10.0,NaN,81.61,3.09,60.0,NaN,NaN,...,NaN,NaN,NaN,NaN,scattered,ENE,100762.47,101226.5,NaN,NaN


In [183]:
df_raw.describe()

,altimeter_set_1,air_temp_set_1,dew_point_temperature_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,sea_level_pressure_set_1,weather_cond_code_set_1,cloud_layer_3_code_set_1,...,air_temp_high_24_hour_set_1,air_temp_low_24_hour_set_1,peak_wind_direction_set_1,dew_point_temperature_set_1d,cloud_layer_1_set_1d,cloud_layer_3_set_1d,cloud_layer_2_set_1d,wind_chill_set_1d,pressure_set_1d,heat_index_set_1d
count,327376.000000,326818.000000,30343.000000,326449.000000,326603.000000,325055.000000,3473.000000,19041.000000,40126.000000,23861.000000,...,1092.000000,1092.000000,306.000000,326448.000000,0.0,0.0,0.0,7414.000000,327376.000000,5974.000000
mean,101515.269021,17.316820,10.832703,68.769009,3.303028,183.979357,10.716936,101523.465679,309.723994,787.708227,...,21.590476,13.759341,244.052288,10.711832,NaN,NaN,NaN,8.342909,101057.289290,28.074560
std,368.914544,4.038202,5.762661,18.465526,2.184637,104.472120,2.576050,386.030869,1586.270444,642.293944,...,4.165966,3.692497,71.033937,5.846914,NaN,NaN,NaN,4.645105,367.250134,1.516821
min,99966.140000,4.000000,-20.600000,0.070000,0.000000,0.000000,7.200000,99940.000000,1.000000,34.000000,...,11.700000,3.900000,10.000000,-58.310000,NaN,NaN,NaN,-3.700000,99515.150000,25.530000
25%,101252.970000,14.000000,8.900000,60.510000,2.060000,90.000000,8.750000,101260.000000,7.000000,354.000000,...,18.900000,11.100000,250.000000,8.000000,NaN,NaN,NaN,4.940000,100796.170000,26.970000
50%,101456.160000,17.000000,12.200000,72.600000,3.090000,240.000000,10.290000,101490.000000,31.000000,553.000000,...,21.700000,13.900000,260.000000,11.970000,NaN,NaN,NaN,8.150000,100998.450000,27.830000
75%,101760.930000,20.000000,15.000000,82.270000,4.630000,260.000000,12.350000,101780.000000,31.000000,1003.000000,...,23.900000,16.700000,270.000000,14.980000,NaN,NaN,NaN,11.330000,101301.840000,28.890000
max,102844.570000,37.000000,21.100000,100.000000,18.520000,360.000000,21.090000,102830.000000,199445.000000,3006.000000,...,37.200000,22.800000,360.000000,21.110000,NaN,NaN,NaN,25.800000,102380.590000,36.560000


In [184]:
## Convert GTM to Local DateTime
shift_hour = -8
from datetime import timedelta
df_raw['date_time_gmt'] = df_raw['date_time']
df_raw['date_time_gmt'] = pd.to_datetime(df_raw['date_time'])
df_raw['date_time'] = df_raw['date_time_gmt'] + timedelta(hours=shift_hour)

In [185]:
# get year day hour
df_raw["year"]= df_raw['date_time'].dt.year 
df_raw["month"]= df_raw['date_time'].dt.month 
df_raw["day"]= df_raw['date_time'].dt.day 
df_raw["hour"]= df_raw['date_time'].dt.hour 
df_raw.head()

,station_id,date_time,altimeter_set_1,air_temp_set_1,dew_point_temperature_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,sea_level_pressure_set_1,...,wind_cardinal_direction_set_1d,pressure_set_1d,sea_level_pressure_set_1d,heat_index_set_1d,weather_condition_set_1d,date_time_gmt,year,month,day,hour
0,KLAX,2018-12-31 04:00:00+00:00,101252.97,9.0,NaN,81.48,1.54,70.0,NaN,NaN,...,ENE,100796.17,101262.01,NaN,NaN,2018-12-31 12:00:00+00:00,2018,12,31,4
1,KLAX,2018-12-31 04:05:00+00:00,101252.97,9.0,NaN,81.48,2.06,60.0,NaN,NaN,...,ENE,100796.17,101262.01,NaN,NaN,2018-12-31 12:05:00+00:00,2018,12,31,4
2,KLAX,2018-12-31 04:10:00+00:00,101252.97,9.0,NaN,81.48,2.06,60.0,NaN,NaN,...,ENE,100796.17,101262.01,NaN,NaN,2018-12-31 12:10:00+00:00,2018,12,31,4
3,KLAX,2018-12-31 04:15:00+00:00,101252.97,9.0,NaN,81.48,2.57,60.0,NaN,NaN,...,ENE,100796.17,101262.01,NaN,NaN,2018-12-31 12:15:00+00:00,2018,12,31,4
4,KLAX,2018-12-31 04:20:00+00:00,101219.11,10.0,NaN,81.61,3.09,60.0,NaN,NaN,...,ENE,100762.47,101226.5,NaN,NaN,2018-12-31 12:20:00+00:00,2018,12,31,4


# Find year round

In [186]:
# TODO: automate the process for multiple years
year = 2021
df_year = df_raw.loc[df_raw['year'] == year]
df_year.head(5)

,station_id,date_time,altimeter_set_1,air_temp_set_1,dew_point_temperature_set_1,relative_humidity_set_1,wind_speed_set_1,wind_direction_set_1,wind_gust_set_1,sea_level_pressure_set_1,...,wind_cardinal_direction_set_1d,pressure_set_1d,sea_level_pressure_set_1d,heat_index_set_1d,weather_condition_set_1d,date_time_gmt,year,month,day,hour
213706,KLAX,2021-01-01 00:00:00+00:00,101557.75,13.0,NaN,37.97,1.54,40.0,NaN,NaN,...,NE,101099.58,101560.28,NaN,NaN,2021-01-01 08:00:00+00:00,2021,1,1,0
213707,KLAX,2021-01-01 00:05:00+00:00,101557.75,11.0,NaN,46.59,1.54,30.0,NaN,NaN,...,NNE,101099.58,101563.52,NaN,NaN,2021-01-01 08:05:00+00:00,2021,1,1,0
213708,KLAX,2021-01-01 00:10:00+00:00,101557.75,11.0,NaN,46.59,0.00,0.0,NaN,NaN,...,N,101099.58,101563.52,NaN,NaN,2021-01-01 08:10:00+00:00,2021,1,1,0
213709,KLAX,2021-01-01 00:15:00+00:00,101557.75,11.0,NaN,46.59,0.00,0.0,NaN,NaN,...,N,101099.58,101563.52,NaN,NaN,2021-01-01 08:15:00+00:00,2021,1,1,0
213710,KLAX,2021-01-01 00:20:00+00:00,101557.75,11.0,NaN,46.59,0.00,0.0,NaN,NaN,...,N,101099.58,101563.52,NaN,NaN,2021-01-01 08:20:00+00:00,2021,1,1,0


In [187]:
average_data_per_hour = len(df_year['date_time']) / 8760
average_data_per_hour = max(int(round(average_data_per_hour, 1)), 1)
average_data_per_hour

13

In [188]:
df_proc = pd.DataFrame()
df_proc['date_time'] = df_year['date_time']
df_proc['month'] = df_year['month']
df_proc['day'] = df_year['day']
df_proc['hour'] = df_year['hour']
for field_name in fields:
    if fields[field_name] in df_year:
        df_proc[field_name] = df_year[fields[field_name]]

In [189]:
len(df_proc['date_time'])

113688

In [190]:
if 'cloud_layer_1' in df_proc:
    df_proc['cloud_layer_1'] = df_proc['cloud_layer_1'].fillna(0)
    df_proc['cloud_layer_1_none'] = np.where((df_proc['cloud_layer_1'] == 0) | (df_proc['cloud_layer_1'] == 1), 1, 0)

### Clean up

In [191]:
df_proc.describe()

,month,day,hour,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_none
count,113688.000000,113688.000000,113688.000000,113497.000000,113131.000000,113131.000000,113672.000000,113540.000000,113061.000000,2611.000000,1179.000000,113688.000000,113688.000000
mean,6.573587,15.650253,11.490967,16.630151,70.644387,10.475064,101084.112575,3.286956,185.057358,8.011528,27.222070,106.040954,0.499006
std,3.431562,8.804679,6.922496,3.952236,18.853950,5.666659,365.153551,2.204038,103.723639,4.833262,0.955945,248.592543,0.500001
min,1.000000,1.000000,0.000000,4.000000,0.070000,-58.310000,99885.980000,0.000000,0.000000,-3.700000,25.660000,0.000000,0.000000
25%,4.000000,8.000000,5.000000,14.000000,63.180000,7.950000,100829.890000,1.540000,90.000000,4.620000,26.480000,1.000000,0.000000
50%,7.000000,16.000000,11.000000,17.000000,75.670000,11.960000,101065.860000,3.090000,240.000000,8.150000,27.030000,6.000000,0.000000
75%,10.000000,23.000000,17.000000,19.000000,82.770000,14.980000,101335.550000,4.630000,260.000000,11.680000,27.700000,126.000000,1.000000
max,12.000000,31.000000,23.000000,33.000000,100.000000,21.000000,102279.460000,18.520000,360.000000,23.450000,30.730000,3303.000000,1.000000


In [192]:
for field_name in fields_to_interp:
    df_proc[field_name] = df_proc[field_name].interpolate(method='linear', limit=average_data_per_hour)
df_proc.describe()

,month,day,hour,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_none
count,113688.000000,113688.000000,113688.000000,113688.000000,113676.000000,113676.000000,113688.000000,113688.000000,113061.000000,2611.000000,1179.000000,113688.000000,113688.000000
mean,6.573587,15.650253,11.490967,16.631199,70.696493,10.478090,101084.108821,3.285929,185.057358,8.011528,27.222070,106.040954,0.499006
std,3.431562,8.804679,6.922496,3.950053,18.855781,5.667262,365.138450,2.203543,103.723639,4.833262,0.955945,248.592543,0.500001
min,1.000000,1.000000,0.000000,4.000000,0.070000,-58.310000,99885.980000,0.000000,0.000000,-3.700000,25.660000,0.000000,0.000000
25%,4.000000,8.000000,5.000000,14.000000,63.180000,7.950000,100829.890000,1.540000,90.000000,4.620000,26.480000,1.000000,0.000000
50%,7.000000,16.000000,11.000000,17.000000,75.840000,11.960000,101065.860000,3.090000,240.000000,8.150000,27.030000,6.000000,0.000000
75%,10.000000,23.000000,17.000000,19.000000,82.770000,14.980000,101335.550000,4.630000,260.000000,11.680000,27.700000,126.000000,1.000000
max,12.000000,31.000000,23.000000,33.000000,100.000000,21.000000,102279.460000,18.520000,360.000000,23.450000,30.730000,3303.000000,1.000000


In [193]:
if 'cloud_layer_1' in df_proc:
    df_proc['cloud_layer_1_digit'] = df_proc['cloud_layer_1'].astype(int) % 10
    df_proc['cloud_layer_1_digit_m'] = np.where(df_proc['cloud_layer_1_none'] == 1, np.nan, df_proc['cloud_layer_1_digit'].astype(int))

In [194]:
if 'weather_condition' in df_proc:
    df_proc['weather_condition'].unique()

In [195]:
if 'weather_condition' in df_proc:
    df_proc['is_rain'] = np.where((df_proc['weather_condition'].str.contains('Rain')) |
                                  (df_proc['weather_condition'].str.contains('Thunder')) |
                                  (df_proc['weather_condition'].str.contains('Drizzle')), 1, 0)

In [196]:
df_proc.head(5)

,date_time,month,day,hour,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_none,cloud_layer_1_digit,cloud_layer_1_digit_m
213706,2021-01-01 00:00:00+00:00,1,1,0,13.0,37.97,-1.1,101099.58,1.54,40.0,NaN,NaN,86.0,0,6,6.0
213707,2021-01-01 00:05:00+00:00,1,1,0,11.0,46.59,-0.1,101099.58,1.54,30.0,NaN,NaN,86.0,0,6,6.0
213708,2021-01-01 00:10:00+00:00,1,1,0,11.0,46.59,-0.1,101099.58,0.00,0.0,NaN,NaN,86.0,0,6,6.0
213709,2021-01-01 00:15:00+00:00,1,1,0,11.0,46.59,-0.1,101099.58,0.00,0.0,NaN,NaN,86.0,0,6,6.0
213710,2021-01-01 00:20:00+00:00,1,1,0,11.0,46.59,-0.1,101099.58,0.00,0.0,NaN,NaN,86.0,0,6,6.0


In [197]:
df_proc.to_csv(os.path.join(process_folder, station_name + '-' + str(year) + '-p.csv'))

# Average to hour

In [198]:
columns = list(fields.keys())
if 'cloud_layer_1' in df_proc:
    columns.append('cloud_layer_1_digit_m')
if 'is_rain' in df_proc:
    columns.append('is_rain')
days_in_mon = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

In [199]:
columns

['dry_bulb',
 'rel_humi',
 'dew_point',
 'pressure',
 'wind_speed',
 'wind_dir',
 'wind_chill',
 'heat_index',
 'cloud_layer_1',
 'cloud_layer_1_digit_m']

In [200]:
df_hourly = pd.DataFrame()
df_hourly['date_time'] = []
for col in columns:
    if col != 'weather_condition':
        df_hourly[col] = []
for m in range(1, 13):
    print('Month: ' + str(m)) 
    for d in range(1, days_in_mon[m] + 1):
        for h in range(24):
            month = str(m).zfill(2)
            day = str(d).zfill(2)
            hour = str(h).zfill(2)
            row_values = list()
            row_values.append(str(year) + '-' + month + '-' + day + '-' + hour + ':00')
            df_loc = df_proc.loc[(df_proc['month'] == int(month)) & (df_proc['day'] == int(day)) & (df_proc['hour'] == int(hour))]
            df_loc_mean = df_loc.mean(axis=0, numeric_only=True)
            df_loc_mode = df_loc.mode(axis=0)
            for col in columns:
                if col != 'weather_condition' and col != 'is_rain':
                    row_values.append(df_loc_mean[col])
            if 'is_rain' in df_loc_mode:
                if len(df_loc_mode['is_rain']) > 0:
                    row_values.append(df_loc_mode['is_rain'][0])
                else:
                    row_values.append(0)
            row_series = pd.Series(row_values, index = df_hourly.columns)
            df_hourly = df_hourly.append(row_series, ignore_index=True)

Month: 1


C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourl

Month: 2


C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourl

Month: 3


...

C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourl

Month: 11


C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourl

Month: 12


C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourly = df_hourly.append(row_series, ignore_index=True)
C:\Users\trent\AppData\Local\Temp\ipykernel_45436\1290788201.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hourl

In [201]:
df_hourly.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8755.000000,8755.000000,8755.000000,8755.000000,8755.000000,8751.000000,433.000000,124.000000,8755.000000,7252.000000
mean,16.637122,70.514956,10.438925,101084.166579,3.276335,184.657037,8.885002,27.067619,109.371366,4.917166
std,3.925774,18.673207,5.632747,364.828531,2.079100,88.750814,4.724245,0.844548,146.646012,1.307495
min,5.661538,6.509231,-14.206923,99901.538462,0.000000,0.000000,-2.258182,25.920000,0.153846,2.000000
25%,13.991987,62.766923,7.934615,100829.890000,1.661923,94.615385,5.800000,26.395385,17.153846,4.000000
50%,16.700000,74.892308,11.473077,101060.676154,2.865714,237.692308,8.809000,26.872587,83.000000,6.000000
75%,19.030769,83.551291,14.637308,101327.771538,4.708462,256.666667,12.541538,27.544808,154.769231,6.000000
max,32.015385,100.000000,18.883846,102279.460000,15.869231,360.000000,23.150000,29.785385,2506.000000,6.000000


In [202]:
for col in columns:
    if col != 'weather_condition':
        df_hourly[col] = df_hourly[col].interpolate(method='linear', limit=3, limit_direction ='both')
df_hourly.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,1195.000000,298.000000,8760.000000,8116.000000
mean,16.637519,70.505083,10.436586,101084.049535,3.276197,184.632748,9.560933,26.827114,109.578066,5.016515
std,3.925650,18.677667,5.632478,364.844277,2.078780,88.731054,4.545214,0.724041,147.022863,1.283298
min,5.661538,6.509231,-14.206923,99901.538462,0.000000,0.000000,-2.258182,25.920000,0.153846,2.000000
25%,13.992147,62.742500,7.934615,100829.890000,1.662115,94.615385,6.486164,26.270768,17.153846,4.000000
50%,16.700000,74.881923,11.468077,101060.676154,2.865714,237.692308,9.721250,26.667821,83.071795,6.000000
75%,19.030769,83.545000,14.636346,101327.771538,4.708462,256.488095,13.039998,27.164751,154.769231,6.000000
max,32.015385,100.000000,18.883846,102279.460000,15.869231,360.000000,23.150000,29.785385,2506.000000,6.000000


In [203]:
result_folder = 'station-result'
df_hourly.to_csv(os.path.join(result_folder, station_name + '-' + str(year) + '.csv'))

## Find nearest Airport Data - Optional

In [204]:
CSV_FILE = 'station-result/' + airport_station_name + '-' + str(year) + '.csv'
CSV_FILE_CLEAN =  'station-result/' + airport_station_name + '-' + str(year) + '-M.csv'
df_csv_result = pd.read_csv(CSV_FILE, sep=',', index_col=0, parse_dates=True, infer_datetime_format=True, encoding='UTF-8')

In [205]:
df_csv_result['cloud_layer_1_digit_m'] = df_csv_result['cloud_layer_1_digit_m'].fillna(method='ffill')
df_csv_result['cloud_layer_1_digit_m'] = df_csv_result['cloud_layer_1_digit_m'].fillna(method='bfill').astype(int)
if 'snow_depth' in df_csv_result:
    df_csv_result['snow_depth'] = df_csv_result['snow_depth'].fillna(0)

In [206]:
df_csv_result.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,1195.000000,298.000000,8760.000000,8760.000000
mean,16.637519,70.505083,10.436586,101084.049535,3.276197,184.632748,9.560933,26.827114,109.578066,4.939954
std,3.925650,18.677667,5.632478,364.844277,2.078780,88.731054,4.545214,0.724041,147.022863,1.392919
min,5.661538,6.509231,-14.206923,99901.538462,0.000000,0.000000,-2.258182,25.920000,0.153846,2.000000
25%,13.992147,62.742500,7.934615,100829.890000,1.662115,94.615385,6.486164,26.270768,17.153846,4.000000
50%,16.700000,74.881923,11.468077,101060.676154,2.865714,237.692308,9.721250,26.667821,83.071795,6.000000
75%,19.030769,83.545000,14.636346,101327.771538,4.708462,256.488095,13.039998,27.164751,154.769231,6.000000
max,32.015385,100.000000,18.883846,102279.460000,15.869231,360.000000,23.150000,29.785385,2506.000000,6.000000


In [207]:
df_csv_result.to_csv(CSV_FILE_CLEAN)

## Fill missing values with the nearest airport (optional)

In [208]:
file_name = os.path.join(result_folder, airport_station_name + '-' + str(year) + '.csv')
df_airport = pd.read_csv(file_name, sep=',', index_col=0, parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
df_airport.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,1195.000000,298.000000,8760.000000,8116.000000
mean,16.637519,70.505083,10.436586,101084.049535,3.276197,184.632748,9.560933,26.827114,109.578066,5.016515
std,3.925650,18.677667,5.632478,364.844277,2.078780,88.731054,4.545214,0.724041,147.022863,1.283298
min,5.661538,6.509231,-14.206923,99901.538462,0.000000,0.000000,-2.258182,25.920000,0.153846,2.000000
25%,13.992147,62.742500,7.934615,100829.890000,1.662115,94.615385,6.486164,26.270768,17.153846,4.000000
50%,16.700000,74.881923,11.468077,101060.676154,2.865714,237.692308,9.721250,26.667821,83.071795,6.000000
75%,19.030769,83.545000,14.636346,101327.771538,4.708462,256.488095,13.039998,27.164751,154.769231,6.000000
max,32.015385,100.000000,18.883846,102279.460000,15.869231,360.000000,23.150000,29.785385,2506.000000,6.000000


In [209]:
file_name = os.path.join(result_folder, station_name + '-' + str(year) + '.csv')
df_station = pd.read_csv(file_name, sep=',', index_col=0, parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
df_station.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,1195.000000,298.000000,8760.000000,8116.000000
mean,16.637519,70.505083,10.436586,101084.049535,3.276197,184.632748,9.560933,26.827114,109.578066,5.016515
std,3.925650,18.677667,5.632478,364.844277,2.078780,88.731054,4.545214,0.724041,147.022863,1.283298
min,5.661538,6.509231,-14.206923,99901.538462,0.000000,0.000000,-2.258182,25.920000,0.153846,2.000000
25%,13.992147,62.742500,7.934615,100829.890000,1.662115,94.615385,6.486164,26.270768,17.153846,4.000000
50%,16.700000,74.881923,11.468077,101060.676154,2.865714,237.692308,9.721250,26.667821,83.071795,6.000000
75%,19.030769,83.545000,14.636346,101327.771538,4.708462,256.488095,13.039998,27.164751,154.769231,6.000000
max,32.015385,100.000000,18.883846,102279.460000,15.869231,360.000000,23.150000,29.785385,2506.000000,6.000000


In [210]:
for index, row in df_station.iterrows():
    for col in fields_to_interp:
        if pd.isna(row[col]):
            df_station.loc[index, col] = df_station.loc[index - 1, col] + (df_airport[col][index] - df_airport[col][index - 1])
df_station['wind_dir'] = df_station['wind_dir'].interpolate(method='linear', limit_direction ='both')
# use_airport = ['snow_depth', 'cloud_layer_1_digit_m', 'is_rain']
use_airport = ['cloud_layer_1_digit_m']
for field in use_airport:
    df_station[field] = df_airport[field]
df_station.describe()

,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,1195.000000,298.000000,8760.000000,8116.000000
mean,16.637519,70.505083,10.436586,101084.049535,3.276197,184.632748,9.560933,26.827114,109.578066,5.016515
std,3.925650,18.677667,5.632478,364.844277,2.078780,88.731054,4.545214,0.724041,147.022863,1.283298
min,5.661538,6.509231,-14.206923,99901.538462,0.000000,0.000000,-2.258182,25.920000,0.153846,2.000000
25%,13.992147,62.742500,7.934615,100829.890000,1.662115,94.615385,6.486164,26.270768,17.153846,4.000000
50%,16.700000,74.881923,11.468077,101060.676154,2.865714,237.692308,9.721250,26.667821,83.071795,6.000000
75%,19.030769,83.545000,14.636346,101327.771538,4.708462,256.488095,13.039998,27.164751,154.769231,6.000000
max,32.015385,100.000000,18.883846,102279.460000,15.869231,360.000000,23.150000,29.785385,2506.000000,6.000000


In [177]:
df_station.to_csv(os.path.join(result_folder, station_name + '-' + str(year) + '-M.csv'))